# Ingest Space Station
Demonstrates how to call a REST API to read data and store result in Azure storage.

In [0]:
# Assign variables to clarify inputs to the spark.conf.set() call.
my_scope = "Fall2025SecretScope"   # Databricks secret scope.
my_key = "assign1store"             # Key vaault secret containing storage account access key.
storage_end_point = "assign1store.dfs.core.windows.net"  # Storage account uri.
container_name = "misc"    # Container name.

# The following spark configuration call uses the variables set above.
spark.conf.set(
    "fs.azure.account.key." + storage_end_point,
    dbutils.secrets.get(scope=my_scope, key=my_key))

# To set the URI to be used in the code below, the container name (assign-1-blob) in the string.
uri = "abfss://" + container_name + "@" + storage_end_point + "/" 
print(uri)


In [0]:
import requests

# Get current space station position
position_api_response = requests.get("http://api.open-notify.org/iss-now.json")

print(position_api_response.content)


In [0]:
import json

# Get current number of people in space
api_response = requests.get(
    "http://api.open-notify.org/astros.json" 
)

# Parse the result to JSON.
json_data = json.loads(api_response.content)

print(json_data)


In [0]:
# Convert the people list to a` dataframe.
if json_data['message']=="success":
    astro_count = json_data['number']
    astros_df = spark.createDataFrame(json_data['people'])

print("Total astronauts:  " + str(astro_count))
display(astros_df)


In [0]:
# Save the dataframe in a few different formats.
astros_df.coalesce(1).write.mode("overwrite").options(header=True).csv(uri + "Bronze/ISS/CSV")
astros_df.write.mode("overwrite").parquet(uri + "Bronze/ISS/Parquet")
astros_df.write.mode("overwrite").format("delta").save(uri + "Bronze/ISS/Delta")

In [0]:
# Read back in.
astros_csv_df = spark.read.csv(uri + "Bronze/ISS/CSV", header=True)
astros_parquet_df = spark.read.parquet(uri + "Bronze/ISS/Parquet")
astros_delta_df = spark.read.format("delta").load(uri + "Bronze/ISS/Delta")

display(astros_csv_df)
display(astros_parquet_df)      
display(astros_delta_df)